In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
test_data = pd.read_pickle("test_data_Lenskit2")
all_recs_joined = pd.read_pickle("all_recs_joined_Lenskit2")

In [5]:
def evaluate_genre(genre): 
    
    providers = list(all_recs_joined.iloc[:,8:])
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    rla.add_metric("APCR")
    rla.add_metric("nd_APCR")
    rla.add_metric("equal_ex")
    rla.add_metric("ndcg")
    results= rla.compute(all_recs_joined, test_data, genre, providers)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [6]:
testproviders = ["Drama", "Action"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

Drama


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


done Drama
Action
done Action


In [8]:
output = pd.concat(allResults, keys=testproviders)
output

rND       rKL       rRD      APCR   nd_APCR  equal_ex  \
       Algorithm                                                               
Drama  ALS        0.083380  0.013984  0.042242  0.983647  0.890658  1.029319   
       FunkSVD    0.066989  0.008333  0.031939  0.990847  0.888038  1.043871   
       ItemItem   0.082374  0.012988  0.047054  0.985265  0.896410  1.014131   
       UserUser   0.080723  0.014304  0.108482  0.978903  0.886109  0.973930   
Action ALS        0.081128  0.013418  0.047109  0.983647  0.890658  1.015144   
       FunkSVD    0.086451  0.019700  0.085875  0.990847  0.888038  0.981281   
       ItemItem   0.073721  0.013510  0.075166  0.985265  0.896410  0.989026   
       UserUser   0.067168  0.011331  0.119873  0.978903  0.886109  0.967057   

                      ndcg  
       Algorithm            
Drama  ALS        0.223507  
       FunkSVD    0.159965  
       ItemItem   0.197654  
       UserUser   0.160600  
Action ALS        0.223507  
       FunkSVD    0.159965  
       ItemItem   0.197654  
       UserUser   0.160600